# 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 정리
1. seoul_filtered_cafe 전처리 (가은언니 코드)
2. 변수 선택 > seoul_cafe (new) 생성
  - 제외 : '시군구코드'  '내외국인구분' '국적'
  - 추가 : '영업자시작일' '소재지시작일' '폐업일자'  (영업자시작일 결측치 처리하는데 필요할까봐 소재지시작일 포함)
3. 결측치 처리 및 변수정리 - 영업자시작일, **소재지도로명**
4. 동일커피점(저가-프랜차이즈) 이름 정리  
5. 고가/저가 프랜차이즈 행 나누기
6. 주소 > 업소명 완성 (메가커피 --점)
7. '3년이내폐업여부' 변수 생성
  + 도로명 주소 결측치 처리 작업
8. 데이터 내보내기

[데이터셋 이름]
- seoul_filtered_cafe
- seoul_cafe = data
- data_clean
- data_clean2
- cafe_low

### 1. seoul_filtered_cafe 전처리

In [ ]:
seoul_filtered_cafe = pd.read_csv('/content/drive/MyDrive/seoul_filtered_cafe.csv',encoding='utf-8')


## 면적에 대한 변수 -> '총면적'변수로 대체, 결측치 0으로 채운 뒤 합 계산
## 진열대, 창고 면적은 전부 결측치라 제외
seoul_filtered_cafe['총면적'] = seoul_filtered_cafe[['영업장조리장면적(㎡)', '영업장객실면적(㎡)', '영업장무도장면적(㎡)',
                    '영업장기타면적(㎡)', '업소내화장실면적(㎡)', '타업소공동화장실면적(㎡)',
                    '영업장탈의실면적(㎡)', '영업장객석면적(㎡)', '작업장면적(㎡)',
                    '검사실면적(㎡)']].fillna(0).sum(axis=1)

# 두 열 중 큰 값을 선택하여 새로운 열에 할당
seoul_filtered_cafe['최대면적'] = np.maximum(seoul_filtered_cafe['총면적'], seoul_filtered_cafe['영업장면적(㎡)'])

# 날짜 -> numeric
seoul_filtered_cafe['영업자시작일'] = pd.to_numeric(seoul_filtered_cafe['영업자시작일'], errors='coerce')
seoul_filtered_cafe['소재지시작일'] = pd.to_numeric(seoul_filtered_cafe['소재지시작일'], errors='coerce')

## 층수 -> numeric
seoul_filtered_cafe['지상_부터'] = pd.to_numeric(seoul_filtered_cafe['지상_부터'], errors='coerce')
seoul_filtered_cafe['지상_까지'] = pd.to_numeric(seoul_filtered_cafe['지상_까지'], errors='coerce')
seoul_filtered_cafe['지하_부터'] = pd.to_numeric(seoul_filtered_cafe['지하_부터'], errors='coerce')
seoul_filtered_cafe['지하_까지'] = pd.to_numeric(seoul_filtered_cafe['지하_까지'], errors='coerce')

### 2. 변수 정리 - seoul_cafe 생성

In [ ]:
# 변수 정리
## 제외 : '시군구코드' '내외국인구분' '국적'
## 추가 : '영업자시작일' '소재지시작일' '폐업일자'
seoul_cafe = seoul_filtered_cafe[['년도','허가신고일','영업자시작일','소재지시작일','업소명','소재지도로명','소재지지번','행정동명','업태명','총면적','지상_부터','지상_까지','지하_부터','지하_까지','총층수','업소위치','모범음식점여부','폐업일자','폐업구분']]

In [ ]:
# csv파일로 내보내기
file_path = '/content/drive/MyDrive/seoul_cafe.csv'
seoul_cafe.to_csv(file_path, index=False)

In [ ]:
seoul_cafe.head(3)

,년도,허가신고일,영업자시작일,소재지시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
0,2019,20191118,20191118.0,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1,2019,20190214,20190214.0,20190214.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
2,2019,20190214,20190520.0,20190214.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
#seoul_cafe = pd.read_csv('/content/drive/MyDrive/seoul_cafe.csv')
data = seoul_cafe.copy()

# 범주형변수로
data['년도'] = data['년도'].astype(object)
data['영업자시작일'] = data['영업자시작일'].astype(object)
data['소재지시작일'] = data['소재지시작일'].astype(object)

In [ ]:
data[data['소재지도로명'] == '서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)']

,년도,허가신고일,영업자시작일,소재지시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
1,2019,20190214,20190214.0,20190214.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
2,2019,20190214,20190520.0,20190214.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
data.isna().sum()

,0
년도,0
허가신고일,0
영업자시작일,21
소재지시작일,22
업소명,0
소재지도로명,0
소재지지번,5
행정동명,5
업태명,26
총면적,0


### 3. 결측치 처리 및 변수정리

#### 영업자시작일 결측치 처리




In [ ]:
data[data['영업자시작일'].isna()]

,년도,허가신고일,영업자시작일,소재지시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
3822,2020,20200120,NaN,NaN,빽다방(굽은다리역점),"서울특별시 강동구 양재대로 1576, 삼익빌딩 1층 (명일동)",서울특별시 강동구 명일동 346번지 1호 삼익빌딩,20200120,NaN,73.00,NaN,1.0,1.0,NaN,NaN,NaN,NaN,명일제1동,NaN
3823,2020,20200107,NaN,NaN,메가엠지씨커피 시흥사거리점,"서울특별시 금천구 시흥대로 227, 1층 (시흥동)",서울특별시 금천구 시흥동 994번지 7호,20200107,NaN,70.40,NaN,1.0,1.0,NaN,NaN,NaN,NaN,시흥제1동,NaN
3824,2020,20200217,NaN,NaN,메가엠지씨커피 성수역점,"서울특별시 성동구 아차산로 113, 1층 104호 (성수동2가)",서울특별시 성동구 성수동2가 300번지 1호 삼진빌딩,20200217,NaN,44.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,성수2가제1동,NaN
3825,2020,20200106,NaN,1101110000000.0,(주)커피빈코리아 하나은행을지로본점,"서울특별시 중구 을지로 35, 하나은행 을지로본점 24층 내 스마트워크센터 (을지로1가)",서울특별시 중구 을지로1가 101번지 1호 하나은행 을지로본점 24층 내 스마트...,20200106,NaN,32.88,NaN,24.0,24.0,NaN,NaN,NaN,NaN,명동,NaN
3826,2020,20200120,NaN,NaN,메가커피 신곡시장점,"서울특별시 양천구 신월로 175, 1층 (신월동)",서울특별시 양천구 신월동 537번지 8호 1층,20200120,NaN,92.73,NaN,1.0,1.0,NaN,NaN,NaN,NaN,신월4동,NaN
3828,2020,20200110,NaN,NaN,메가엠지씨커피군자역점,"서울특별시 광진구 능동로 294, 능동빌딩 1층 (능동)",서울특별시 광진구 능동 279번지 4호 능동빌딩,20200110,NaN,50.80,NaN,1.0,1.0,NaN,NaN,NaN,NaN,능동,NaN
3829,2020,20200110,NaN,NaN,메가엠지씨커피군자역점,"서울특별시 광진구 능동로 294, 능동빌딩 1층 (능동)",서울특별시 광진구 능동 279번지 4호 능동빌딩,20200110,NaN,50.80,NaN,1.0,1.0,NaN,NaN,NaN,NaN,능동,NaN
3830,2020,20200204,NaN,NaN,메가커피신대방삼거리역점,"서울특별시 동작구 보라매로 113, 코스모빌딩 104호 (대방동)",서울특별시 동작구 대방동 405번지 11호 코스모빌딩,20200204,NaN,59.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대방동,NaN
3831,2020,20200203,NaN,NaN,빽다방구로KT점,"서울특별시 관악구 남부순환로 1377, 1층 2호 (신림동)",서울특별시 관악구 신림동 1668번지 15호,20200203,NaN,56.20,NaN,1.0,1.0,NaN,NaN,NaN,NaN,조원동,NaN
3832,2020,20200128,NaN,NaN,투썸플레이스 장안동사거리점,"서울특별시 동대문구 답십리로 273, 한양빌딩 1층 (장안동)",서울특별시 동대문구 장안동 307번지 6호 한양빌딩,20200128,NaN,207.90,NaN,1.0,1.0,NaN,NaN,NaN,NaN,장안제2동,NaN


In [ ]:
# 영업자시작일 결측치 허가신고일로 채움
data['영업자시작일'] = data['영업자시작일'].fillna(data['허가신고일'])

data[data['영업자시작일'].isna()] # 해결

,년도,허가신고일,영업자시작일,소재지시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분


In [ ]:
# 중복행 정리 : '영업자시작일' 가장 최근 것만 남기기
data_clean = data.loc[data.groupby('소재지도로명')['영업자시작일'].idxmax()]

## 이 코드 돌리려면 '영업자시작일'에 결측치 없어야 해서 위에서 채워줌

In [ ]:
data_clean.head(3)

,년도,허가신고일,영업자시작일,소재지시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
4831,2023,20230620,20230620.0,20230707.0,투썸플레이스 양지공사점,,"서울특별시 서초구 내곡동 1번지 1459호 지하1층,1층",내곡동,커피숍,344.43,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
5182,2023,20231123,20231123.0,20231123.0,메가엠지씨커피 가로수길점,"서울특별시 강남구 가로수길 19, 지상1층 (신사동)",서울특별시 강남구 신사동 536번지 18호,신사동,커피숍,42.90,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
986,2017,20170120,20170120.0,20170120.0,매머드커피 강남대로점(114호점),"서울특별시 강남구 강남대로 272, 102동 1층 103호 (도곡동, 도곡푸르지오)",서울특별시 강남구 도곡동 949번지 2호,도곡1동,커피숍,56.37,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
# '허가신고일' '소재지시작일' 변수 지우기
data_clean2 = data.drop(['허가신고일', '소재지시작일'], axis=1)

In [ ]:
data_clean2.head(3)

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
0,2019,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1,2019,20190214.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
2,2019,20190520.0,스타벅스 연합뉴스점,"서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)",서울특별시 종로구 수송동 110번지 연합뉴스빌딩,종로1.2.3.4가동,커피숍,53.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN


#### 소재지도로명 결측치 채우기

In [ ]:
data_clean2[data_clean2['업소명'] == '투썸플레이스 양지공사점']

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
4831,2023,20230620.0,투썸플레이스 양지공사점,,"서울특별시 서초구 내곡동 1번지 1459호 지하1층,1층",내곡동,커피숍,344.43,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
data_clean2.loc[4831]['소재지도로명']

'    '

In [ ]:
data_clean2[data_clean2['소재지도로명'] == '    ']   #212개,,,,,,,,,,,

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
1834,2014,20160711.0,이디야 커피,,"서울특별시 강북구 미아동 860번지 76호 외 1필지 (미아동 860-74,75...",삼양동,커피숍,87.74,1.0,1.0,NaN,NaN,3.0,지상,비대상,NaN,NaN
1861,2014,20141202.0,이디야,,서울특별시 은평구 증산동 223번지 27호 디지털미디어시티역내 A/1구간,증산동,커피숍,60.00,NaN,NaN,1.0,1.0,3.0,지하,비대상,20190926.0,자진폐업
1862,2014,20180129.0,이디야,,서울특별시 은평구 증산동 223번지 27호 디지털미디어시티역내 A/1구간,증산동,커피숍,60.00,NaN,NaN,1.0,1.0,3.0,지하,비대상,20190926.0,자진폐업
2548,2012,20121009.0,이디야 역삼시장점,,서울특별시 강남구 역삼동 796번지 22호 지상106호,역삼1동,다방,35.53,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
2655,2011,20110506.0,메가웹스테이션,,서울특별시 도봉구 창동 75번지 1호 (5층),창제5동,일반조리판매,6.60,5.0,5.0,NaN,NaN,NaN,지상2층이상,비대상,20120515.0,자진폐업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3814,1996,19970103.0,할리스커피,,서울특별시 서초구 양재동 23번지 0호 환승상가111호,양재제1동,다방,76.38,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20050920.0,자진폐업
3815,1994,20010510.0,할리스,,서울특별시 동작구 노량진동 112번지 10호,노량진제1동,과자점,69.16,NaN,NaN,NaN,NaN,NaN,지하,비대상,20030611.0,자진폐업
3819,1992,19950327.0,할리스,,서울특별시 강남구 신사동 661번지 17호,압구정동,다방,52.17,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20000221.0,자진폐업
3821,1983,20021203.0,할리스,,서울특별시 강남구 역삼동 821번지 1호,역삼1동,다방,159.00,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20050909.0,자진폐업


In [ ]:
############## 소재지지번 > 소재지도로명 결측치 채우기 (>좌표)

## 추후에 코드 추가

### 4~5. 저가 프랜차이즈
- 저가 프랜차이즈 이름 통일
- 저가 프랜차이즈 카페 데이터셋만 뽑기
- 업소명 완성 (메가커피 --점)
- '프랜차이즈명' 변수 생성 (메가커피)

#### 1) *메가커피*

##### EDA

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('메가커피')]
#.shape #56

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
55,2019,20200318.0,메가커피 건대학생회관점,"서울특별시 광진구 능동로 120, 건국대학교 학생회관 1층 (화양동)",서울특별시 광진구 화양동 1번지 건국대학교 학생회관1층,화양동,커피숍,20.70,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
56,2019,20190830.0,메가커피 건대학생회관점,"서울특별시 광진구 능동로 120, 건국대학교 학생회관 1층 (화양동)",서울특별시 광진구 화양동 1번지 건국대학교 학생회관1층,화양동,커피숍,20.70,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
84,2019,20190214.0,메가커피 상계역점,"서울특별시 노원구 덕릉로83길 18, 1층 102호 (중계동)",서울특별시 노원구 중계동 154번지 22호,중계4동,커피숍,30.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
114,2019,20190507.0,메가커피(북가좌점),"서울특별시 서대문구 응암로 108, 지상1층 (북가좌동)",서울특별시 서대문구 북가좌동 292번지 1호,북가좌제2동,커피숍,37.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
163,2019,20190613.0,메가커피 등촌성당점,"서울특별시 강서구 공항대로41길 43, 새한프라자 1층 103호 (등촌동)",서울특별시 강서구 등촌동 696번지 1호 새한프라자 (지상 1층)-103,등촌제3동,커피숍,32.04,1.0,1.0,NaN,NaN,9.0,지상,비대상,NaN,NaN
328,2019,20190729.0,메가커피,"서울특별시 강동구 올림픽로 783, 102호 (암사동)",서울특별시 강동구 암사동 502번지 21호,암사제2동,커피숍,36.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
358,2018,20200727.0,메가커피,"서울특별시 중구 다산로 162, 약수빌딩 1층 101호 (신당동)",서울특별시 중구 신당동 340번지 73호 약수빌딩 1층 101호,청구동,커피숍,19.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
372,2018,20181002.0,메가커피,"서울특별시 중구 다산로 162, 약수빌딩 1층 101호 (신당동)",서울특별시 중구 신당동 340번지 73호 약수빌딩 1층 101호,청구동,커피숍,19.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
408,2018,20180319.0,MEGA COFFEE 메가커피 동소문점,"서울특별시 성북구 아리랑로 14, 1층 (동선동4가)",서울특별시 성북구 동선동4가 42번지,동선동,커피숍,27.80,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
418,2018,20190424.0,메가커피 미아수유시장점,"서울특별시 강북구 솔매로45길 95, 1층 (미아동)",서울특별시 강북구 미아동 217번지 109호 1층,미아동,커피숍,62.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('메가엠지씨')]
#.shape #548

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
27,2019,20190531.0,메가엠지씨커피 동대문점,"서울특별시 중구 장충단로13길 13, 1층 (을지로6가)",서울특별시 중구 을지로6가 18번지 97호,광희동,커피숍,15.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
39,2019,20200122.0,메가엠지씨커피 금호역점,"서울특별시 성동구 장터길 30, 1층 (금호동4가)",서울특별시 성동구 금호동4가 1202번지,금호4가동,커피숍,80.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
40,2019,20191104.0,메가엠지씨커피 금호역점,"서울특별시 성동구 장터길 30, 1층 (금호동4가)",서울특별시 성동구 금호동4가 1202번지,금호4가동,커피숍,80.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
45,2019,20191001.0,메가엠지씨커피 답십리로데오점,"서울특별시 성동구 용답중앙15길 12, 1층 (용답동)",서울특별시 성동구 용답동 14번지 4호,용답동,커피숍,14.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
50,2019,20190603.0,메가엠지씨커피,"서울특별시 광진구 능동로 403-1, 1층 (중곡동)",서울특별시 광진구 중곡동 166번지 9호,중곡제3동,커피숍,99.30,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5522,2024,20240111.0,메가엠지씨커피 금천구립시흥도서관점,"서울특별시 금천구 금하로 750, 관악우방아파트 상가1동 108호 (시흥동)",서울특별시 금천구 시흥동 1011번지,시흥제2동,커피숍,51.75,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
5530,2024,20240105.0,메가엠지씨커피 공항시장역점,"서울특별시 강서구 방화동로 49-1, 1층 (방화동)",서울특별시 강서구 방화동 614번지 60호,방화제2동,커피숍,70.47,1.0,1.0,NaN,NaN,3.0,지상,비대상,NaN,NaN
5533,2024,20240129.0,메가엠지씨커피 도봉산입구점,"서울특별시 도봉구 도봉산4길 4-13, 1층,2층 (도봉동)",서울특별시 도봉구 도봉동 280번지 4호,도봉제1동,커피숍,193.36,1.0,2.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
5535,2024,20240104.0,메가엠지씨커피 청량리롯데캐슬점,"서울특별시 동대문구 답십리로 27, 1층 109호 (전농동, 청량리역 롯데캐슬 SK...",서울특별시 동대문구 전농동 620번지 47호 청량리역 롯데캐슬 SKY-L65,전농제1동,커피숍,29.70,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('메가MGC')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
412,2018,20180716.0,메가MGC(엠지씨)커피월곡두산점,"서울특별시 성북구 오패산로 46, 주상가동 1층 106호 (하월곡동, 월곡두산위브아파트)",서울특별시 성북구 하월곡동 222번지 월곡두산위브아파트,길음제2동,커피숍,32.76,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
1177,2016,20160418.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1182,2016,20180306.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1186,2016,20181226.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
4978,2023,20230515.0,메가MGC커피 서강대점,"서울특별시 마포구 백범로 34, 1층 (신수동)",서울특별시 마포구 신수동 81번지 77호,대흥동,커피숍,46.05,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
5097,2023,20230823.0,메가MGC커피 독산대륭20차점,"서울특별시 금천구 가산디지털1로 5, 대륭테크노타운 20차 지상1층 115호 (가산동)",서울특별시 금천구 가산동 691번지 대륭테크노타운 20차,가산동,커피숍,45.36,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
5103,2023,20230831.0,메가MGC커피 구로대륭포스트2차점,"서울특별시 구로구 디지털로 306, 대륭포스트타워2차 1층 107호 (구로동)",서울특별시 구로구 구로동 182번지 13호 대륭포스트타워2차,구로제3동,커피숍,67.77,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('MGC')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
412,2018,20180716.0,메가MGC(엠지씨)커피월곡두산점,"서울특별시 성북구 오패산로 46, 주상가동 1층 106호 (하월곡동, 월곡두산위브아파트)",서울특별시 성북구 하월곡동 222번지 월곡두산위브아파트,길음제2동,커피숍,32.76,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
414,2018,20180531.0,MEGA MGC COFFEE 메가엠지씨커피 한성대입구역점,"서울특별시 성북구 동소문로 9, 1층 (동소문동1가)",서울특별시 성북구 동소문동1가 27번지,성북동,커피숍,52.89,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
415,2018,20190813.0,MEGA MGC COFFEE 메가엠지씨커피 한성대입구역점,"서울특별시 성북구 동소문로 9, 1층 (동소문동1가)",서울특별시 성북구 동소문동1가 27번지,성북동,커피숍,52.89,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
1177,2016,20160418.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1182,2016,20180306.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
1186,2016,20181226.0,메가MGC커피 상암 사보이시티점,"서울특별시 마포구 월드컵북로54길 17, 1층 130호 (상암동, 사보이시티디엠씨)",서울특별시 마포구 상암동 1597번지,상암동,기타 휴게음식점,11.97,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
4256,2021,20230629.0,메가엠지씨커피 마곡역점(MEGA MGC COFFEE),"서울특별시 강서구 공항대로 190, 푸리마타워 1층 107호 (마곡동)",서울특별시 강서구 마곡동 799번지 2호 푸리마타워 1층-107,발산제1동,커피숍,48.94,1.0,1.0,NaN,NaN,18.0,지상,비대상,NaN,NaN
4978,2023,20230515.0,메가MGC커피 서강대점,"서울특별시 마포구 백범로 34, 1층 (신수동)",서울특별시 마포구 신수동 81번지 77호,대흥동,커피숍,46.05,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
5097,2023,20230823.0,메가MGC커피 독산대륭20차점,"서울특별시 금천구 가산디지털1로 5, 대륭테크노타운 20차 지상1층 115호 (가산동)",서울특별시 금천구 가산동 691번지 대륭테크노타운 20차,가산동,커피숍,45.36,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
5103,2023,20230831.0,메가MGC커피 구로대륭포스트2차점,"서울특별시 구로구 디지털로 306, 대륭포스트타워2차 1층 107호 (구로동)",서울특별시 구로구 구로동 182번지 13호 대륭포스트타워2차,구로제3동,커피숍,67.77,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('MEGA')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
408,2018,20180319.0,MEGA COFFEE 메가커피 동소문점,"서울특별시 성북구 아리랑로 14, 1층 (동선동4가)",서울특별시 성북구 동선동4가 42번지,동선동,커피숍,27.80,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
414,2018,20180531.0,MEGA MGC COFFEE 메가엠지씨커피 한성대입구역점,"서울특별시 성북구 동소문로 9, 1층 (동소문동1가)",서울특별시 성북구 동소문동1가 27번지,성북동,커피숍,52.89,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
415,2018,20190813.0,MEGA MGC COFFEE 메가엠지씨커피 한성대입구역점,"서울특별시 성북구 동소문로 9, 1층 (동소문동1가)",서울특별시 성북구 동소문동1가 27번지,성북동,커피숍,52.89,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
4256,2021,20230629.0,메가엠지씨커피 마곡역점(MEGA MGC COFFEE),"서울특별시 강서구 공항대로 190, 푸리마타워 1층 107호 (마곡동)",서울특별시 강서구 마곡동 799번지 2호 푸리마타워 1층-107,발산제1동,커피숍,48.94,1.0,1.0,NaN,NaN,18.0,지상,비대상,NaN,NaN
4829,2023,20230328.0,메가후르츠(MEGAFRUITS)장승배기점,"서울특별시 동작구 장승배기로10길 23, 1층 (상도동)",서울특별시 동작구 상도동 373번지 34호,상도제2동,기타 휴게음식점,32.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN
4896,2023,20230216.0,메가후르츠(MEGAFRUITS)장승배기점,"서울특별시 동작구 장승배기로10길 23, 1층 (상도동)",서울특별시 동작구 상도동 373번지 34호,상도제2동,기타 휴게음식점,32.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,20230315.0,기타
5468,2024,20240305.0,메가엠지씨커피(MEGA MGC COFFEE) 신월시영점,"서울특별시 양천구 지양로 2, 1층 (신월동)",서울특별시 양천구 신월동 994번지 3호,신월7동,커피숍,99.17,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('mega')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분
4129,2021,20211125.0,카페오메가(cafe omega),"서울특별시 강동구 명일로 375, 삼익그린맨션 1층 18호 (명일동)",서울특별시 강동구 명일동 15번지 삼익그린맨션,명일제1동,커피숍,4.95,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN
5124,2023,20230918.0,메가후르츠(megafruits),"서울특별시 성북구 성북로23길 9, 지상1층 (성북동)",서울특별시 성북구 성북동 114번지 8호,성북동,커피숍,217.80,1.0,1.0,NaN,NaN,NaN,지상,비대상,20231208.0,자진폐업


##### 전처리

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('메가.*커피|커피.*메가', regex=True)]
#data_clean2[data_clean2['업소명'].str.contains('메가') & data_clean2['업소명'].str.contains('커피')]
#622개
data_clean2.loc[data_clean2['업소명'].str.contains('메가.*커피|커피.*메가', regex=True), '프랜차이즈명'] = '메가커피'
print(data_clean2.head())

     년도      영업자시작일          업소명                                      소재지도로명  \
0  2019  20191118.0       컴포즈 커피            서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)   
1  2019  20190214.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
2  2019  20190520.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
3  2019  20190129.0  투썸플레이스 경복궁점  서울특별시 종로구 율곡로 6, 트윈 트리 빌딩 A동 1층 102호 (중학동)   
4  2019  20190902.0          이디야                 서울특별시 종로구 성균관로 8, 1층 (명륜2가)   

                               소재지지번         행정동명       업태명     총면적  지상_부터  \
0       서울특별시 종로구 계동  140번지 2호  현대빌딩          가회동       커피숍   16.44    NaN   
1       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
2       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
3  서울특별시 종로구 중학동  14번지  트윈 트리 빌딩-102  종로1.2.3.4가동  기타 휴게음식점  208.37    NaN   
4         서울특별시 종로구 명륜2가  209번지 2호            혜화동       커피숍   84.18    NaN   

   지상_까지  지하_부터  지하_까지  총층수 업소위치 모범음식점여부        폐업

##### 가짜 메가커피 제거

- 메가후르츠
카페오메가
메가웹 메가플 메가존 메가박 메가디 메가존 메가푸 메가몬 메가진 메가보 메가샵 메가후 메가감 메가혼 메가런
- 오메가피씨방 오메가라이더카페
메가푸드앤서비스주식회사노량진지점
씨제이
주식회사 메가젠임플란트 메가젠타워지점
메가박스씨너스주식회사
주식회사 떡메가




#### (2) 컴포즈

##### EDA

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('컴포즈')]['업소명'].unique()

array(['컴포즈 커피', '컴포즈커피 성수역점', '컴포즈커피 수락산역점', '컴포즈커피', '컴포즈커피 마곡리더스퀘어점',
       '컴포즈커피 가양역점', '컴포즈커피 방화역점', '컴포즈커피 구로구청점', '컴포즈커피 구로한영시티점',
       '컴포즈커피 가산디지털단지역점', '컴포즈커피 가산로데오점', '컴포즈 에이스가산타워점', '컴포즈',
       '컴포즈커피 가산스카이밸리점', '컴포즈커피영등포구청한솔빌딩점', '컴포즈커피 성대시장점', '컴포즈커피 봉천점',
       '컴포즈커피 논현영동시장점', '컴포즈커피 잠실역 지하점', '컴포즈커피 성수점', '컴포즈커피 목동점',
       '컴포즈커피 마곡나루점', '컴포즈커피 가산1호점', '컴포즈커피여의도라이프점', '컴포즈커피 문정테라타워점',
       '컴포즈커피 마포점', '컴포즈커피 신사점', '컴포즈커피 잠실점', '컴포즈커피 종로구청점',
       '컴포즈커피 은행사거리점', '컴포즈커피(상암점)', '컴포즈커피 합정점', '컴포즈커피(호서대점)',
       '컴포즈 오류북부점', '컴포즈커피 신림대학점', '컴포즈커피 구로디지털2호점',
       '컴포즈커피(compose coffee) 구로디지털2호점', '컴포즈커피이수역점', '컴포즈커피대림성모병원점',
       '컴포즈커피중랑구청점', '컴포즈커피오목교점', '컴포즈커피(사당점)', '컴포즈커피 강남역사점',
       '컴포즈커피 서소문점', '컴포즈 교보타워점', '컴포즈커피 (구로디지털점)', '컴포즈커피중곡점',
       '컴포즈커피 신대방삼거리역점', '컴포즈 은평갈현연서점', '컴포즈커피 남성역점', '컴포즈커피 역촌점',
       '컴포즈커피 송파사거리점', '컴포즈커피 독산대륭테크노타운점', '컴포즈커피 은평신사점',
       '컴포즈커피(홍대 동교동점)', '컴포즈 신월양강점', '컴포즈커피 개봉한진점', '컴포즈 커피 개봉시장점',
       '컴포즈커피 노원

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('컴포즈')].shape
# 396

(396, 18)

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('컴포즈 커피')].shape

(8, 18)

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('컴포즈커피')].shape

(372, 18)

##### 전처리

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('컴포즈', regex=True)]
#396

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
0,2019,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
44,2019,20190416.0,컴포즈커피 성수역점,"서울특별시 성동구 아차산로 96, 1층 102호 (성수동2가)",서울특별시 성동구 성수동2가 314번지 6호 신성빌딩,성수2가제1동,커피숍,43.65,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
95,2019,20191028.0,컴포즈커피 수락산역점,"서울특별시 노원구 동일로 1689, 1층 103호 (상계동, 하이베라스)",서울특별시 노원구 상계동 1262번지 하이베라스,상계1동,커피숍,39.19,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
113,2019,20191105.0,컴포즈커피,"서울특별시 서대문구 홍은중앙로 107, (홍은동)",서울특별시 서대문구 홍은동 9번지 317호,홍은제1동,기타 휴게음식점,70.03,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
157,2019,20190805.0,컴포즈커피 마곡리더스퀘어점,"서울특별시 강서구 마곡중앙6로 45, 리더스퀘어마곡 1층 128호 (마곡동)",서울특별시 강서구 마곡동 772번지 5호 리더스퀘어마곡 1층-128,가양제1동,커피숍,38.35,1.0,1.0,NaN,NaN,9.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5525,2024,20240110.0,컴포즈커피 하계타운점,"서울특별시 노원구 한글비석로1길 67, 1층 좌측호 (하계동)",서울특별시 노원구 하계동 213번지 1호 1층-좌측,하계1동,커피숍,83.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5527,2024,20240108.0,컴포즈커피 용산원효로점,"서울특별시 용산구 원효로41길 23, 1층 (원효로3가)",서울특별시 용산구 원효로3가 31번지 2호,원효로제2동,기타 휴게음식점,24.00,1.0,1.0,NaN,NaN,2.0,지상,비대상,NaN,NaN,nan
5532,2024,20240105.0,컴포즈커피 송파풍납점,"서울특별시 송파구 토성로 65, 태양상가 1층 110,111호 (풍납동)",서울특별시 송파구 풍납동 406번지 1호 태양상가,풍납2동,커피숍,50.40,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5536,2024,20240104.0,컴포즈커피 양재이마트점,"서울특별시 서초구 매헌로 16, 하이브랜드 지하1층 리빙관 지102호 (양재동)",서울특별시 서초구 양재동 215번지 하이브랜드 지하1층 리빙관 지102호,양재제2동,기타 휴게음식점,27.81,NaN,NaN,1.0,1.0,NaN,지하,비대상,NaN,NaN,nan


In [ ]:
data_clean2.loc[data_clean2['업소명'].str.contains('컴포즈', regex=True), '프랜차이즈명'] = '컴포즈커피'
print(data_clean2.head())

     년도      영업자시작일          업소명                                      소재지도로명  \
0  2019  20191118.0       컴포즈 커피            서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)   
1  2019  20190214.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
2  2019  20190520.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
3  2019  20190129.0  투썸플레이스 경복궁점  서울특별시 종로구 율곡로 6, 트윈 트리 빌딩 A동 1층 102호 (중학동)   
4  2019  20190902.0          이디야                 서울특별시 종로구 성균관로 8, 1층 (명륜2가)   

                               소재지지번         행정동명       업태명     총면적  지상_부터  \
0       서울특별시 종로구 계동  140번지 2호  현대빌딩          가회동       커피숍   16.44    NaN   
1       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
2       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
3  서울특별시 종로구 중학동  14번지  트윈 트리 빌딩-102  종로1.2.3.4가동  기타 휴게음식점  208.37    NaN   
4         서울특별시 종로구 명륜2가  209번지 2호            혜화동       커피숍   84.18    NaN   

   지상_까지  지하_부터  지하_까지  총층수 업소위치 모범음식점여부        폐업

#### (3) 매머드

##### EDA

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('매머드|메머드')] #377

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
16,2019,20190410.0,매머드 익스프레스 청계다동점,"서울특별시 중구 다동길 43, 한외빌딩 1층 118,119,120호 (다동)",서울특별시 중구 다동 70번지 한외빌딩,명동,커피숍,33.33,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
30,2019,20190401.0,매머드 익스프레스 동대문점,"서울특별시 중구 장충단로13길 43, 1층 24-1호 (을지로6가)",서울특별시 중구 을지로6가 18번지 187호,광희동,커피숍,6.45,1.0,1.0,NaN,NaN,1.0,지상,비대상,NaN,NaN,nan
51,2019,20190422.0,매머드익스프레스 건대후문점,"서울특별시 광진구 광나루로24길 29, 1층 (화양동)",서울특별시 광진구 화양동 428번지 1호 1층,화양동,커피숍,4.90,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
108,2019,20190617.0,매머드익스프레스 이대점,"서울특별시 서대문구 이화여대8길 2, 무궁화상가아파트 1동 제1층 101호 (대현동)",서울특별시 서대문구 대현동 54번지 1호 무궁화상가아파트,신촌동,기타 휴게음식점,18.88,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
109,2019,20200422.0,매머드익스프레스 이대점,"서울특별시 서대문구 이화여대8길 2, 무궁화상가아파트 1동 제1층 101호 (대현동)",서울특별시 서대문구 대현동 54번지 1호 무궁화상가아파트,신촌동,기타 휴게음식점,18.88,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5500,2024,20240214.0,매머드익스프레스 영등포한양아이클래스점,"서울특별시 영등포구 영신로 228-6, 한양아이클래스오피스텔 1층 107호 (영등포...",서울특별시 영등포구 영등포동8가 42번지 1호 한양아이클래스오피스텔,영등포동,커피숍,37.69,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5516,2024,20240119.0,매머드커피,"서울특별시 중랑구 용마산로90길 9, 천광빌딩 (면목동)",서울특별시 중랑구 면목동 19번지 58호 천광빌딩,면목제3.8동,커피숍,26.00,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
5517,2024,20240117.0,매머드 익스프레스 한양대병원점,"서울특별시 성동구 사근동길 4, 대학약국 1층 (행당동)",서울특별시 성동구 행당동 14번지 1호 대학약국,행당제1동,커피숍,33.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5523,2024,20240111.0,매머드커피 방이2호점,"서울특별시 송파구 오금로13길 3-11, 한길빌딩 1층 102호 (방이동)",서울특별시 송파구 방이동 65번지 7호 한길빌딩,방이2동,커피숍,40.26,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('매머드|메머드')]['업소명'].unique()

array(['매머드 익스프레스 청계다동점', '매머드 익스프레스 동대문점', '매머드익스프레스 건대후문점',
       '매머드익스프레스 이대점', '매머드커피 이대점', '매머드 익스프레스 등촌역점',
       '매머드 익스프레스 마곡그랑트윈타워점', '매머드 익스프레스 화곡메가박스점', '매머드 익스프레스 까치산역점',
       '매머드 익스프레스 구로대명벨리온점', '매머드 익스프레스 구로 삼성아이티밸리점', '매머드커피 구로대명벨리온점',
       '매머드 익스프레스 가산더스카이밸리 1차점', '매머드익스프레스 시흥동점', '매머드 익스프레스 갑을그레이트밸리점',
       '매머드익스프레스양평역점', '매머드익스프레스(여의도공원점)', '매머드커피 상도역점', '매머드익스프레스 숭실대점',
       '매머드커피', '매머드 익스프레스 이수역점', '주식회사 매머드커피랩', '매머드 익스프레스 강남오거리점',
       '매머드익스프레스 (경찰병원점)', '매머드 익스프레스 문정테라타워', '매머드 익스프레스 충무로역점',
       '매머드 익스프레스 남산점', '매머드커피 남산점', '매머드커피 장교빌딩점',
       '매머드익스프레스(MAMMOTH EXPRESS) 성수우림', '매머드커피사가정점',
       '매머드커피(MAMMOTH COFFEE)마포합정점', '매머드커피(MAMMOTH COFFEE)',
       '매머드익스프레스 우림블루나인(38호)', '매머드커피한국노총점', '매머드익스프레스(당산중앙점)',
       '매머드 익스프레스 서초법원점', '매머드 익스프레스 수서로즈데일점', '매머드 익스프레스(논현원&원점)',
       '매머드익스프레스 선릉한신점', '매머드커피익스프레스 뱅뱅사거리점', '매머드익스프레스논현 원앤원점25호',
       '(주)매머드커피랩', '매머드 익스프레스 문정SKV1', '매머드커피(장한비바패밀리점)',
       '매머드익스프레스외대점(mammothexpres

##### 전처리

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('매머드|메머드', regex=True)]
# 377

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
16,2019,20190410.0,매머드 익스프레스 청계다동점,"서울특별시 중구 다동길 43, 한외빌딩 1층 118,119,120호 (다동)",서울특별시 중구 다동 70번지 한외빌딩,명동,커피숍,33.33,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
30,2019,20190401.0,매머드 익스프레스 동대문점,"서울특별시 중구 장충단로13길 43, 1층 24-1호 (을지로6가)",서울특별시 중구 을지로6가 18번지 187호,광희동,커피숍,6.45,1.0,1.0,NaN,NaN,1.0,지상,비대상,NaN,NaN,nan
51,2019,20190422.0,매머드익스프레스 건대후문점,"서울특별시 광진구 광나루로24길 29, 1층 (화양동)",서울특별시 광진구 화양동 428번지 1호 1층,화양동,커피숍,4.90,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
108,2019,20190617.0,매머드익스프레스 이대점,"서울특별시 서대문구 이화여대8길 2, 무궁화상가아파트 1동 제1층 101호 (대현동)",서울특별시 서대문구 대현동 54번지 1호 무궁화상가아파트,신촌동,기타 휴게음식점,18.88,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
109,2019,20200422.0,매머드익스프레스 이대점,"서울특별시 서대문구 이화여대8길 2, 무궁화상가아파트 1동 제1층 101호 (대현동)",서울특별시 서대문구 대현동 54번지 1호 무궁화상가아파트,신촌동,기타 휴게음식점,18.88,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5500,2024,20240214.0,매머드익스프레스 영등포한양아이클래스점,"서울특별시 영등포구 영신로 228-6, 한양아이클래스오피스텔 1층 107호 (영등포...",서울특별시 영등포구 영등포동8가 42번지 1호 한양아이클래스오피스텔,영등포동,커피숍,37.69,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5516,2024,20240119.0,매머드커피,"서울특별시 중랑구 용마산로90길 9, 천광빌딩 (면목동)",서울특별시 중랑구 면목동 19번지 58호 천광빌딩,면목제3.8동,커피숍,26.00,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
5517,2024,20240117.0,매머드 익스프레스 한양대병원점,"서울특별시 성동구 사근동길 4, 대학약국 1층 (행당동)",서울특별시 성동구 행당동 14번지 1호 대학약국,행당제1동,커피숍,33.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5523,2024,20240111.0,매머드커피 방이2호점,"서울특별시 송파구 오금로13길 3-11, 한길빌딩 1층 102호 (방이동)",서울특별시 송파구 방이동 65번지 7호 한길빌딩,방이2동,커피숍,40.26,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2.loc[data_clean2['업소명'].str.contains('매머드|메머드', regex=True), '프랜차이즈명'] = '매머드커피'
print(data_clean2.head())

     년도      영업자시작일          업소명                                      소재지도로명  \
0  2019  20191118.0       컴포즈 커피            서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)   
1  2019  20190214.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
2  2019  20190520.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
3  2019  20190129.0  투썸플레이스 경복궁점  서울특별시 종로구 율곡로 6, 트윈 트리 빌딩 A동 1층 102호 (중학동)   
4  2019  20190902.0          이디야                 서울특별시 종로구 성균관로 8, 1층 (명륜2가)   

                               소재지지번         행정동명       업태명     총면적  지상_부터  \
0       서울특별시 종로구 계동  140번지 2호  현대빌딩          가회동       커피숍   16.44    NaN   
1       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
2       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
3  서울특별시 종로구 중학동  14번지  트윈 트리 빌딩-102  종로1.2.3.4가동  기타 휴게음식점  208.37    NaN   
4         서울특별시 종로구 명륜2가  209번지 2호            혜화동       커피숍   84.18    NaN   

   지상_까지  지하_부터  지하_까지  총층수 업소위치 모범음식점여부        폐업

#### (4) 더벤티

##### EDA

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('더벤티')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
65,2019,20190522.0,더벤티 성신여대점,"서울특별시 성북구 동소문로22길 6, 1층 (동선동1가)",서울특별시 성북구 동선동1가 1번지 26호,동선동,커피숍,16.53,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
106,2019,20190521.0,더벤티서대문역점,"서울특별시 서대문구 통일로 147, 1층 (냉천동)",서울특별시 서대문구 냉천동 237번지 1층,천연동,다방,32.59,1.0,1.0,NaN,NaN,2.0,지상,비대상,NaN,NaN,nan
148,2019,20190114.0,더벤티 마곡점,"서울특별시 강서구 마곡중앙6로 42, 사이언스타 1층 123호 (마곡동)",서울특별시 강서구 마곡동 774번지 9호 사이언스타-123,가양제1동,커피숍,57.33,1.0,1.0,NaN,NaN,17.0,지상,비대상,NaN,NaN,nan
150,2019,20190927.0,더벤티 까치산역점,"서울특별시 강서구 강서로 52, 화곡판타지아 1층 108호 (화곡동)",서울특별시 강서구 화곡동 341번지 65호 화곡판타지아(지상 1층)-108,화곡제8동,커피숍,34.72,1.0,1.0,NaN,NaN,10.0,지상,비대상,NaN,NaN,nan
165,2019,20191122.0,더벤티 마곡역점,"서울특별시 강서구 마곡서로 56, 우성에스비타워3 1층 104호 (마곡동)",서울특별시 강서구 마곡동 798번지 4호 (지상 1층)-104,발산제1동,커피숍,50.54,1.0,1.0,NaN,NaN,13.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,2024,20240221.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 109,110호 (목동)","서울특별시 양천구 목동 615번지 1호 -109,110",목3동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5494,2024,20240216.0,더벤티(신내역시티프라디움점),"서울특별시 중랑구 용마산로 670, C142호 (망우동, 신내역 프라디움 더 테라스)",서울특별시 중랑구 망우동 584번지 신내역 프라디움 더 테라스,망우본동,커피숍,36.28,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5503,2024,20240205.0,더벤티 구로대륭포스트8차점,"서울특별시 구로구 디지털로26길 43, 1층 106호 (구로동)",서울특별시 구로구 구로동 235번지 3호,구로제3동,커피숍,59.51,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
5504,2024,20240202.0,더벤티 방이시장점,"서울특별시 송파구 백제고분로48길 9, 산수빌딩 1층 103호 (방이동)",서울특별시 송파구 방이동 145번지 35호 산수빌딩,방이2동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('벤티')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
65,2019,20190522.0,더벤티 성신여대점,"서울특별시 성북구 동소문로22길 6, 1층 (동선동1가)",서울특별시 성북구 동선동1가 1번지 26호,동선동,커피숍,16.53,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
106,2019,20190521.0,더벤티서대문역점,"서울특별시 서대문구 통일로 147, 1층 (냉천동)",서울특별시 서대문구 냉천동 237번지 1층,천연동,다방,32.59,1.0,1.0,NaN,NaN,2.0,지상,비대상,NaN,NaN,nan
148,2019,20190114.0,더벤티 마곡점,"서울특별시 강서구 마곡중앙6로 42, 사이언스타 1층 123호 (마곡동)",서울특별시 강서구 마곡동 774번지 9호 사이언스타-123,가양제1동,커피숍,57.33,1.0,1.0,NaN,NaN,17.0,지상,비대상,NaN,NaN,nan
150,2019,20190927.0,더벤티 까치산역점,"서울특별시 강서구 강서로 52, 화곡판타지아 1층 108호 (화곡동)",서울특별시 강서구 화곡동 341번지 65호 화곡판타지아(지상 1층)-108,화곡제8동,커피숍,34.72,1.0,1.0,NaN,NaN,10.0,지상,비대상,NaN,NaN,nan
165,2019,20191122.0,더벤티 마곡역점,"서울특별시 강서구 마곡서로 56, 우성에스비타워3 1층 104호 (마곡동)",서울특별시 강서구 마곡동 798번지 4호 (지상 1층)-104,발산제1동,커피숍,50.54,1.0,1.0,NaN,NaN,13.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,2024,20240221.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 109,110호 (목동)","서울특별시 양천구 목동 615번지 1호 -109,110",목3동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5494,2024,20240216.0,더벤티(신내역시티프라디움점),"서울특별시 중랑구 용마산로 670, C142호 (망우동, 신내역 프라디움 더 테라스)",서울특별시 중랑구 망우동 584번지 신내역 프라디움 더 테라스,망우본동,커피숍,36.28,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5503,2024,20240205.0,더벤티 구로대륭포스트8차점,"서울특별시 구로구 디지털로26길 43, 1층 106호 (구로동)",서울특별시 구로구 구로동 235번지 3호,구로제3동,커피숍,59.51,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
5504,2024,20240202.0,더벤티 방이시장점,"서울특별시 송파구 백제고분로48길 9, 산수빌딩 1층 103호 (방이동)",서울특별시 송파구 방이동 145번지 35호 산수빌딩,방이2동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('벤티')]['업소명'].unique()

array(['더벤티 성신여대점', '더벤티서대문역점', '더벤티 마곡점', '더벤티 까치산역점', '더벤티 마곡역점',
       '더벤티 중앙대점', '더벤티', '더벤티킴스클럽목동점', '더벤티목동파리공원점', '더벤티 롯데마트김포공항점',
       '더벤티 가산대륭20차', '더벤티 롯데마트본사점', '더벤티 잠실장미상가점', '더벤티 천호로데오점',
       '더벤티 발산역점', '더벤티 NC강서점', '더벤티서여의도점', '더벤티노량진점', '더벤티강남점',
       '더벤티 동대문역점', '더벤티(서울대입구역점)', '더벤티 삼성점', '더벤티코리아', '더벤티커피전문점',
       '더벤티 신촌점', '더벤티 뱅뱅사거리점', '더벤티(봉천역점)', '더벤티 응암오거리점', '더벤티 신월사거리점',
       '더벤티 개화산역점', '더벤티 공릉점', '더벤티 길동점', '더벤티 대흥역점', '더벤티 마포프레스티지자이점',
       '더벤티 연신내점', '더벤티 논현역점', '더벤티창동역점', '더벤티 혜화로점', '더벤티 구로항동점',
       '더벤티 항동SKV1점', '더벤티 마포망원점', '더벤티 성대시장점', '더벤티 송파래미안점', '더벤티 양천구청점',
       '더벤티강서구청사거리점', '더벤티 개봉점', '더벤티상계사거리점', '더벤티 방학사거리점', '더벤티 개포점',
       '더벤티 장안벤처빌점', '더벤티 마들역점', '더벤티 명일역점', '더벤티 신정네거리역점',
       '더벤티 노원문화의거리점', '더벤티 답십리사거리점', '더벤티 가재울뉴타운', '더벤티 중계은행사거리점',
       '더벤티방학역점', '더벤티 고덕역점', '더벤티 석촌점', '더벤티 사가정아이파크점', '더벤티 상계불암점',
       '더벤티 장한평역점', '더벤티 중화중앙점', '더벤티 공항시장역점', '더벤티 문정점', '더벤티공릉중앙점',
       '더벤티 수유시장점', '더벤티 정릉중앙점', '더

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('밴티')]  # 없음

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('the')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
735,2017,20170224.0,매머드익스프레스외대점(mammothexpress),"서울특별시 동대문구 휘경로 5, 1층 (이문동)",서울특별시 동대문구 이문동 288번지 25호,이문제1동,커피숍,29.9,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피
4613,2022,20221102.0,더벤티커피(theVenti COFFEE) 등촌2동점,"서울특별시 강서구 등촌로 173, 1층 3호 (등촌동)",서울특별시 강서구 등촌동 513번지 1호 -3,등촌제2동,커피숍,27.0,1.0,1.0,NaN,NaN,4.0,지상,비대상,NaN,NaN,nan
4947,2023,20230413.0,the Venti(더벤티) 홍제역점,"서울특별시 서대문구 통일로 472, 효신빌딩 1층 (홍제동)",서울특별시 서대문구 홍제동 299번지 2호 효신빌딩,홍제제3동,커피숍,57.2,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5184,2023,20231124.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 108호 (목동)",서울특별시 양천구 목동 615번지 1호 -108,목3동,커피숍,30.0,NaN,NaN,NaN,NaN,NaN,지상,비대상,20240103.0,자진폐업,nan
5486,2024,20240221.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 109,110호 (목동)","서울특별시 양천구 목동 615번지 1호 -109,110",목3동,커피숍,60.0,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


##### 전처리

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('더벤티', regex=True)]
#164

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
65,2019,20190522.0,더벤티 성신여대점,"서울특별시 성북구 동소문로22길 6, 1층 (동선동1가)",서울특별시 성북구 동선동1가 1번지 26호,동선동,커피숍,16.53,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
106,2019,20190521.0,더벤티서대문역점,"서울특별시 서대문구 통일로 147, 1층 (냉천동)",서울특별시 서대문구 냉천동 237번지 1층,천연동,다방,32.59,1.0,1.0,NaN,NaN,2.0,지상,비대상,NaN,NaN,nan
148,2019,20190114.0,더벤티 마곡점,"서울특별시 강서구 마곡중앙6로 42, 사이언스타 1층 123호 (마곡동)",서울특별시 강서구 마곡동 774번지 9호 사이언스타-123,가양제1동,커피숍,57.33,1.0,1.0,NaN,NaN,17.0,지상,비대상,NaN,NaN,nan
150,2019,20190927.0,더벤티 까치산역점,"서울특별시 강서구 강서로 52, 화곡판타지아 1층 108호 (화곡동)",서울특별시 강서구 화곡동 341번지 65호 화곡판타지아(지상 1층)-108,화곡제8동,커피숍,34.72,1.0,1.0,NaN,NaN,10.0,지상,비대상,NaN,NaN,nan
165,2019,20191122.0,더벤티 마곡역점,"서울특별시 강서구 마곡서로 56, 우성에스비타워3 1층 104호 (마곡동)",서울특별시 강서구 마곡동 798번지 4호 (지상 1층)-104,발산제1동,커피숍,50.54,1.0,1.0,NaN,NaN,13.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,2024,20240221.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 109,110호 (목동)","서울특별시 양천구 목동 615번지 1호 -109,110",목3동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5494,2024,20240216.0,더벤티(신내역시티프라디움점),"서울특별시 중랑구 용마산로 670, C142호 (망우동, 신내역 프라디움 더 테라스)",서울특별시 중랑구 망우동 584번지 신내역 프라디움 더 테라스,망우본동,커피숍,36.28,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5503,2024,20240205.0,더벤티 구로대륭포스트8차점,"서울특별시 구로구 디지털로26길 43, 1층 106호 (구로동)",서울특별시 구로구 구로동 235번지 3호,구로제3동,커피숍,59.51,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
5504,2024,20240202.0,더벤티 방이시장점,"서울특별시 송파구 백제고분로48길 9, 산수빌딩 1층 103호 (방이동)",서울특별시 송파구 방이동 145번지 35호 산수빌딩,방이2동,커피숍,60.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2.loc[data_clean2['업소명'].str.contains('더벤티', regex=True), '프랜차이즈명'] = '더벤티'
print(data_clean2.head())

     년도      영업자시작일          업소명                                      소재지도로명  \
0  2019  20191118.0       컴포즈 커피            서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)   
1  2019  20190214.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
2  2019  20190520.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
3  2019  20190129.0  투썸플레이스 경복궁점  서울특별시 종로구 율곡로 6, 트윈 트리 빌딩 A동 1층 102호 (중학동)   
4  2019  20190902.0          이디야                 서울특별시 종로구 성균관로 8, 1층 (명륜2가)   

                               소재지지번         행정동명       업태명     총면적  지상_부터  \
0       서울특별시 종로구 계동  140번지 2호  현대빌딩          가회동       커피숍   16.44    NaN   
1       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
2       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
3  서울특별시 종로구 중학동  14번지  트윈 트리 빌딩-102  종로1.2.3.4가동  기타 휴게음식점  208.37    NaN   
4         서울특별시 종로구 명륜2가  209번지 2호            혜화동       커피숍   84.18    NaN   

   지상_까지  지하_부터  지하_까지  총층수 업소위치 모범음식점여부        폐업

#### (5) 빽다방

##### EDA

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('빽다방')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
53,2019,20191108.0,빽다방구의역점,"서울특별시 광진구 아차산로 380, 세화빌딩 1층 (자양동)",서울특별시 광진구 자양동 216번지 9호 세화빌딩,자양제2동,커피숍,33.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
59,2019,20190109.0,빽다방,"서울특별시 동대문구 장한로 140, 1층 (장안동)",서울특별시 동대문구 장안동 308번지 1호,장안제1동,커피숍,39.60,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
94,2019,20190503.0,빽다방(노원문화의거리점),"서울특별시 노원구 노해로79길 6-19, 화랑상가 나동 1층 104호 (상계동)",서울특별시 노원구 상계동 617번지 나 화랑상가-104,상계2동,커피숍,42.97,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
97,2019,20190813.0,빽다방 응암대림시장점,"서울특별시 은평구 응암로 169, 1층 (응암동)",서울특별시 은평구 응암동 595번지 1호 1층,응암제1동,커피숍,41.25,1.0,1.0,NaN,NaN,4.0,지상,비대상,NaN,NaN,nan
162,2019,20190709.0,빽다방 화곡역점,"서울특별시 강서구 화곡로25길 2, 1층 106호 (화곡동)",서울특별시 강서구 화곡동 1065번지 1호 -106,화곡제3동,커피숍,41.61,1.0,1.0,NaN,NaN,8.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506,2024,20240201.0,빽다방 강남을지논현점,"서울특별시 강남구 도산대로34길 15, 지상1층 102호 (논현동)",서울특별시 강남구 논현동 64번지 8호,논현1동,커피숍,37.57,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5513,2024,20240124.0,빽다방 대치사거리점,"서울특별시 강남구 삼성로 345, CMS S 지상1층 101호 (대치동)",서울특별시 강남구 대치동 913번지 11호 CMS S,대치1동,커피숍,31.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5515,2024,20240903.0,빽다방 이대점,"서울특별시 서대문구 이화여대길 28, 1층 101,102,103호 (대현동)",서울특별시 서대문구 대현동 54번지 8호,신촌동,커피숍,49.50,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5520,2024,20240116.0,빽다방 방학사거리점,"서울특별시 도봉구 도봉로 679, 1층 (방학동)",서울특별시 도봉구 방학동 703번지 64호,방학제1동,커피숍,58.74,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('빽')]['업소명'].unique()

array(['빽다방구의역점', '빽다방', '빽다방(노원문화의거리점)', '빽다방 응암대림시장점', '빽다방 화곡역점',
       '빽다방 오류동역남부점', '빽다방 동여의도점', '빽다방동여의도점', '빽다방 보라매타운점', '빽다방 신림난곡점',
       '빽다방 봉천역점', '양재시민의숲역 빽다방', '빽다방(방이시장점)', '빽다방 응암역점', '빽다방 KBS본관점',
       '빽다방 신대방역점', '빽다방 신사역점', '빽다방신설동역1호점', '빽다방노원역북부점',
       '빽다방 구파발역 미켈란점', '빽다방 발산역플레이스점', '빽다방 마곡나루역북부점', '빽다방 방이역 코오롱점',
       '빽다방 남대문시장점', '빽다방 서울숲역점', '빽다방청량리역점', '빽다방 중랑역 동부시장점', '빽다방 길음역점',
       '명지대 빽다방', '빽다방망원시장점', '빽다방등촌역점', '빽다방목동41타워점', '빽다방 까치산역점',
       '빽다방-마곡역GMG점', '빽다방-목동사거리점', '빽다방 천왕역점', '빽다방 개봉역점', '빽다방영등포구청역점',
       '빽다방(당산역점)', '빽다방신대방삼거리점', '빽다방 신림역2호점', '빽다방 총신대입구역점',
       '빽다방 고속터미널지하점', '빽다방 역삼GS점', '빽다방 문정법조 현대점', '빽다방 광희동사거리',
       '빽다방명동중앙우체국점', '빽다방 신용산역점', '빽다방 왕십리역점', '광진구청점 빽다방',
       '빽다방(건대로데오점)', '회기역점 빽다방', '빽다방 외대역점', '빽다방(노원역우리은행점)',
       '빽다방 연신내역점', '빽다방(구파발역점)', '빽다방(응암역이마트점)', '빽다방 은평초교점',
       '홍제삼거리점 빽다방', '공덕역롯데캐슬점빽다방', '빽다방상암KGIT점', '빽다방홍대극동방송점',
       "마포복사꽃공원점빽다방PAIK'S COFFEE", '홍대입구점빽다방', '빽다방 목동파

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('PAIK')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
1542,2015,20150615.0,마포복사꽃공원점빽다방PAIK'S COFFEE,"서울특별시 마포구 마포대로4길 16, (도화동, 1층일부)",서울특별시 마포구 도화동 290번지 15호,도화동,다방,56.19,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1708,2015,20150324.0,PAIK'S COFFEE(빽다방),"서울특별시 송파구 올림픽로35길 124, (신천동, 장미A상가 1동 110호)",서울특별시 송파구 신천동 7번지 장미A상가 1동 110호,잠실4동,기타 휴게음식점,35.60,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('Paik')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명


In [ ]:
data_clean2[data_clean2['업소명'].str.contains('paik')]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("PAIK'S")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
1542,2015,20150615.0,마포복사꽃공원점빽다방PAIK'S COFFEE,"서울특별시 마포구 마포대로4길 16, (도화동, 1층일부)",서울특별시 마포구 도화동 290번지 15호,도화동,다방,56.19,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1708,2015,20150324.0,PAIK'S COFFEE(빽다방),"서울특별시 송파구 올림픽로35길 124, (신천동, 장미A상가 1동 110호)",서울특별시 송파구 신천동 7번지 장미A상가 1동 110호,잠실4동,기타 휴게음식점,35.60,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


##### 전처리

In [ ]:
data_clean2[data_clean2['업소명'].str.contains('빽다방', regex=True)]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
53,2019,20191108.0,빽다방구의역점,"서울특별시 광진구 아차산로 380, 세화빌딩 1층 (자양동)",서울특별시 광진구 자양동 216번지 9호 세화빌딩,자양제2동,커피숍,33.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
59,2019,20190109.0,빽다방,"서울특별시 동대문구 장한로 140, 1층 (장안동)",서울특별시 동대문구 장안동 308번지 1호,장안제1동,커피숍,39.60,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
94,2019,20190503.0,빽다방(노원문화의거리점),"서울특별시 노원구 노해로79길 6-19, 화랑상가 나동 1층 104호 (상계동)",서울특별시 노원구 상계동 617번지 나 화랑상가-104,상계2동,커피숍,42.97,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
97,2019,20190813.0,빽다방 응암대림시장점,"서울특별시 은평구 응암로 169, 1층 (응암동)",서울특별시 은평구 응암동 595번지 1호 1층,응암제1동,커피숍,41.25,1.0,1.0,NaN,NaN,4.0,지상,비대상,NaN,NaN,nan
162,2019,20190709.0,빽다방 화곡역점,"서울특별시 강서구 화곡로25길 2, 1층 106호 (화곡동)",서울특별시 강서구 화곡동 1065번지 1호 -106,화곡제3동,커피숍,41.61,1.0,1.0,NaN,NaN,8.0,지상,비대상,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506,2024,20240201.0,빽다방 강남을지논현점,"서울특별시 강남구 도산대로34길 15, 지상1층 102호 (논현동)",서울특별시 강남구 논현동 64번지 8호,논현1동,커피숍,37.57,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5513,2024,20240124.0,빽다방 대치사거리점,"서울특별시 강남구 삼성로 345, CMS S 지상1층 101호 (대치동)",서울특별시 강남구 대치동 913번지 11호 CMS S,대치1동,커피숍,31.35,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5515,2024,20240903.0,빽다방 이대점,"서울특별시 서대문구 이화여대길 28, 1층 101,102,103호 (대현동)",서울특별시 서대문구 대현동 54번지 8호,신촌동,커피숍,49.50,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
5520,2024,20240116.0,빽다방 방학사거리점,"서울특별시 도봉구 도봉로 679, 1층 (방학동)",서울특별시 도봉구 방학동 703번지 64호,방학제1동,커피숍,58.74,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2.loc[data_clean2['업소명'].str.contains('빽다방', regex=True), '프랜차이즈명'] = '빽다방'
print(data_clean2.head())

     년도      영업자시작일          업소명                                      소재지도로명  \
0  2019  20191118.0       컴포즈 커피            서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)   
1  2019  20190214.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
2  2019  20190520.0   스타벅스 연합뉴스점         서울특별시 종로구 율곡로2길 25, 연합뉴스빌딩 1층 (수송동)   
3  2019  20190129.0  투썸플레이스 경복궁점  서울특별시 종로구 율곡로 6, 트윈 트리 빌딩 A동 1층 102호 (중학동)   
4  2019  20190902.0          이디야                 서울특별시 종로구 성균관로 8, 1층 (명륜2가)   

                               소재지지번         행정동명       업태명     총면적  지상_부터  \
0       서울특별시 종로구 계동  140번지 2호  현대빌딩          가회동       커피숍   16.44    NaN   
1       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
2       서울특별시 종로구 수송동  110번지  연합뉴스빌딩  종로1.2.3.4가동       커피숍   53.35    1.0   
3  서울특별시 종로구 중학동  14번지  트윈 트리 빌딩-102  종로1.2.3.4가동  기타 휴게음식점  208.37    NaN   
4         서울특별시 종로구 명륜2가  209번지 2호            혜화동       커피숍   84.18    NaN   

   지상_까지  지하_부터  지하_까지  총층수 업소위치 모범음식점여부        폐업

####(6) 그 외 EDA



In [ ]:
data_clean2[data_clean2['업소명'].str.contains("the")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
735,2017,20170224.0,매머드익스프레스외대점(mammothexpress),"서울특별시 동대문구 휘경로 5, 1층 (이문동)",서울특별시 동대문구 이문동 288번지 25호,이문제1동,커피숍,29.9,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피
4613,2022,20221102.0,더벤티커피(theVenti COFFEE) 등촌2동점,"서울특별시 강서구 등촌로 173, 1층 3호 (등촌동)",서울특별시 강서구 등촌동 513번지 1호 -3,등촌제2동,커피숍,27.0,1.0,1.0,NaN,NaN,4.0,지상,비대상,NaN,NaN,더벤티
4947,2023,20230413.0,the Venti(더벤티) 홍제역점,"서울특별시 서대문구 통일로 472, 효신빌딩 1층 (홍제동)",서울특별시 서대문구 홍제동 299번지 2호 효신빌딩,홍제제3동,커피숍,57.2,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,더벤티
5184,2023,20231124.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 108호 (목동)",서울특별시 양천구 목동 615번지 1호 -108,목3동,커피숍,30.0,NaN,NaN,NaN,NaN,NaN,지상,비대상,20240103.0,자진폐업,더벤티
5486,2024,20240221.0,더벤티(the Venti) 목동깨비시장점,"서울특별시 양천구 목동중앙북로4길 6, 109,110호 (목동)","서울특별시 양천구 목동 615번지 1호 -109,110",목3동,커피숍,60.0,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,더벤티


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("굿커피")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
1790,2014,20161116.0,굿커피빈스,"서울특별시 성동구 자동차시장1길 85, 108호 (용답동)",서울특별시 성동구 용답동 235번지 8호 -108,용답동,커피숍,39.35,NaN,NaN,NaN,NaN,NaN,지상,비대상,20190108.0,자진폐업,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("더커피")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
1934,2014,20141023.0,더커피빈,"서울특별시 영등포구 영중로 15, (영등포동4가, 타임스퀘어 1층 111호)",서울특별시 영등포구 영등포동4가 442번지 타임스퀘어 1층 111호,영등포동,커피숍,58.90,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
2223,2013,20131211.0,더커피빈앤티리프,"서울특별시 영등포구 국회대로74길 19, (여의도동, 동우빌딩 1층 일부)",서울특별시 영등포구 여의도동 13번지 4호 동우빌딩 1층 일부,여의동,커피숍,279.70,1.0,1.0,NaN,NaN,NaN,지상,비대상,20200915.0,자진폐업,nan
2236,2013,20131022.0,더커피빈카페엔샌드위치(The coffeebean cafe&sandwich),"서울특별시 영등포구 의사당대로 97, (여의도동, 교보증권빌딩 2층 일부)",서울특별시 영등포구 여의도동 26번지 4호 교보증권빌딩 2층 일부,여의동,커피숍,297.00,2.0,2.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
3746,2001,20010829.0,(주)커피빈코리아 더커피빈앤티리프,,서울특별시 마포구 서교동 344번지 11호,서교동,다방,149.04,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20090220.0,자진폐업,nan
3770,2001,20180607.0,더커피빈 앤 티리프,"서울특별시 강남구 광평로 280, (수서동,로즈데일빌딩 지상1층)",서울특별시 강남구 수서동 724번지 0호 로즈데일빌딩 지상1층,수서동,다방,163.20,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
3772,2001,20010518.0,더커피빈 앤 티리프,"서울특별시 강남구 광평로 280, (수서동,로즈데일빌딩 지상1층)",서울특별시 강남구 수서동 724번지 0호 로즈데일빌딩 지상1층,수서동,다방,163.20,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,nan
3775,2001,20010423.0,더커피빈 앤 티리프,,"서울특별시 강남구 청담동 85번지 5호 지상1,2층",청담동,다방,313.38,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20061025.0,자진폐업,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("카페베")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
2135,2013,20130319.0,카페베네 상봉메가박스점,"서울특별시 중랑구 망우로30길 3, 1층 111,112,113,114호 (상봉동)","서울특별시 중랑구 상봉동 130번지 3호 111,112,113,114호",상봉제2동,커피숍,188.75,1.0,NaN,NaN,NaN,NaN,지상,비대상,20141119.0,자진폐업,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("A T")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
411,2018,20181211.0,투썸플레이스 보문역점(A TWOSOME PLACE 보문역점),"서울특별시 성북구 보문로 125, 1층 (보문동2가)",서울특별시 성북구 보문동2가 105번지 3호,보문동,커피숍,184.22,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
413,2018,20190911.0,투썸플레이스 보문역점(A TWOSOME PLACE 보문역점),"서울특별시 성북구 보문로 125, 1층 (보문동2가)",서울특별시 성북구 보문동2가 105번지 3호,보문동,커피숍,184.22,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
2267,2013,20180222.0,A TWOSOME PLACE(투썸플레이스) 반포효성점,"서울특별시 서초구 반포대로 235, 1층 (반포동)",서울특별시 서초구 반포동 63번지 7호 1층,반포제4동,커피숍,131.47,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
2268,2013,20131122.0,A TWOSOME PLACE(투썸플레이스) 반포효성점,"서울특별시 서초구 반포대로 235, 1층 (반포동)",서울특별시 서초구 반포동 63번지 7호 1층,반포제4동,커피숍,131.47,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
2702,2011,20150925.0,A TWOSOME PLACE(투썸플레이스),"서울특별시 구로구 경인로 199-2, (오류동)",서울특별시 구로구 오류동 39번지 6호,오류제1동,일반조리판매,159.21,2.0,3.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("카페이")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
3177,2008,20090811.0,카페이디야,"서울특별시 서대문구 거북골로 33, (남가좌동,(지하1층))",서울특별시 서대문구 남가좌동 324번지 35호 (지하1층),남가좌제1동,일반조리판매,131.97,NaN,NaN,1.0,1.0,NaN,지하,비대상,20171031.0,자진폐업,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("주-커")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
3229,2008,20081024.0,주-커피빈코리아 강남역 142호점,"서울특별시 서초구 강남대로65길 7, (서초동,1층(101호))",서울특별시 서초구 서초동 1307번지 18호 1층(101호),서초제1동,커피숍,290.72,1.0,1.0,NaN,NaN,7.0,지상,비대상,20181109.0,자진폐업,nan
3236,2008,20081024.0,주-커피빈코리아 삼성전자 강남사옥점,"서울특별시 서초구 서초대로74길 11, (서초동,지하2층)",서울특별시 서초구 서초동 1320번지 10호 지하2층,서초제1동,커피숍,44.32,NaN,NaN,2.0,2.0,NaN,지하,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("오메가")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
1637,2015,20150513.0,오메가피씨방,"서울특별시 관악구 남부순환로 1832, 지하1층 (봉천동, 오선빌딩)",서울특별시 관악구 봉천동 853번지 4호,낙성대동,일반조리판매,13.80,NaN,NaN,1.0,1.0,14.0,지하,비대상,NaN,NaN,nan
1639,2015,20180123.0,오메가피씨방,"서울특별시 관악구 남부순환로 1832, 지하1층 (봉천동, 오선빌딩)",서울특별시 관악구 봉천동 853번지 4호,낙성대동,일반조리판매,13.80,NaN,NaN,1.0,1.0,14.0,지하,비대상,NaN,NaN,nan
1836,2014,20140409.0,오메가라이더카페,"서울특별시 강북구 덕릉로 136-1, (번동)",서울특별시 강북구 번동 437번지 12호,번2동,커피숍,40.58,1.0,1.0,NaN,NaN,3.0,지상,비대상,NaN,NaN,nan
1837,2014,20140409.0,오메가라이더카페,"서울특별시 강북구 덕릉로 136-1, (번동)",서울특별시 강북구 번동 437번지 12호,번제2동,커피숍,40.58,1.0,1.0,NaN,NaN,3.0,지상,비대상,NaN,NaN,nan
4129,2021,20211125.0,카페오메가(cafe omega),"서울특별시 강동구 명일로 375, 삼익그린맨션 1층 18호 (명일동)",서울특별시 강동구 명일동 15번지 삼익그린맨션,명일제1동,커피숍,4.95,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("CGV")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
153,2019,20190417.0,CGV등촌 투썸플레이스,"서울특별시 강서구 공항대로45길 63, 3층 (등촌동)",서울특별시 강서구 등촌동 639번지 23호 (지상3층),등촌제1동,커피숍,69.29,3.0,3.0,NaN,NaN,13.0,지상,비대상,NaN,NaN,nan
155,2019,20190124.0,CGV등촌 투썸플레이스,"서울특별시 강서구 공항대로45길 63, 3층 (등촌동)",서울특별시 강서구 등촌동 639번지 23호 (지상3층),등촌제1동,커피숍,69.29,3.0,3.0,NaN,NaN,13.0,지상,비대상,NaN,NaN,nan
3049,2009,20120416.0,투썸플레이스CGV영등포점,"서울특별시 영등포구 영중로 15, (영등포동4가,4층 (구 441-10번지 4층))",서울특별시 영등포구 영등포동4가 442번지 4층 (구 441-10번지 4층),영등포동,패스트푸드,190.75,4.0,4.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
3050,2009,20180208.0,투썸플레이스CGV영등포점,"서울특별시 영등포구 영중로 15, (영등포동4가,4층 (구 441-10번지 4층))",서울특별시 영등포구 영등포동4가 442번지 4층 (구 441-10번지 4층),영등포동,패스트푸드,190.75,4.0,4.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
3051,2009,20180112.0,투썸플레이스CGV영등포점,"서울특별시 영등포구 영중로 15, (영등포동4가,4층 (구 441-10번지 4층))",서울특별시 영등포구 영등포동4가 442번지 4층 (구 441-10번지 4층),영등포동,패스트푸드,190.75,4.0,4.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
3052,2009,20190702.0,투썸플레이스CGV영등포점,"서울특별시 영등포구 영중로 15, (영등포동4가,4층 (구 441-10번지 4층))",서울특별시 영등포구 영등포동4가 442번지 4층 (구 441-10번지 4층),영등포동,패스트푸드,190.75,4.0,4.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("(주)")]

<ipython-input-62-fdfc2adac2e7>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_clean2[data_clean2['업소명'].str.contains("(주)")]


,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
168,2019,20190820.0,(주)커피빈코리아 화곡역점,"서울특별시 강서구 강서로 162, 다온빌딩 1층 (화곡동)",서울특별시 강서구 화곡동 1076번지 19호 다온빌딩 1층,화곡본동,커피숍,232.52,1.0,1.0,NaN,NaN,5.0,지상,비대상,NaN,NaN,nan
245,2019,20190509.0,(주)커피빈코리아 교대법원점,"서울특별시 서초구 서초중앙로 156, 1층 (서초동)",서울특별시 서초구 서초동 1699번지 16호 1층,서초제4동,커피숍,178.76,1.0,1.0,NaN,NaN,NaN,지상,비대상,20200911.0,자진폐업,nan
252,2019,20190711.0,(주)커피빈코리아 서초동점,"서울특별시 서초구 반포대로 70, 1층,2층 (서초동)","서울특별시 서초구 서초동 1582번지 12호 1층,2층",서초제1동,커피숍,242.52,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
257,2019,20190806.0,(주)커피빈코리아 행사점,"서울특별시 강남구 영동대로 513, 코엑스 B홀호 1층 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,삼성1동,커피숍,99.00,NaN,NaN,NaN,NaN,NaN,NaN,비대상,20190821.0,자진폐업,nan
258,2019,20190218.0,(주)커피빈코리아 행사점,"서울특별시 강남구 영동대로 513, 코엑스 지상1층 B홀 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스 지상1층 B홀,삼성1동,커피숍,99.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,20190225.0,자진폐업,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,2023,20231106.0,메가엠지씨커피 구로주공점,"서울특별시 구로구 구일로4길 33, 종합상가 102호 (구로동)",서울특별시 구로구 구로동 685번지 215호 종합상가,구로제1동,커피숍,20.45,1.0,1.0,NaN,NaN,NaN,NaN,비대상,NaN,NaN,메가커피
5232,2024,20240912.0,매머드 익스프레스 역삼언주로점,"서울특별시 강남구 언주로85길 32, 역삼역센트럴푸르지오 지상2층 201호 (역삼동)",서울특별시 강남구 역삼동 719번지 24호 역삼역센트럴푸르지오,역삼1동,커피숍,25.89,2.0,2.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피
5266,2024,20240816.0,(주)비지에프휴먼넷 이디야 관악산으뜸점,"서울특별시 관악구 신림로 21, 2층 (신림동)",서울특별시 관악구 신림동 210번지,대학동,커피숍,167.28,2.0,2.0,NaN,NaN,3.0,NaN,비대상,NaN,NaN,nan
5310,2024,20240715.0,매머드 익스프레스 언주역점,"서울특별시 강남구 봉은사로30길 11, 1층 102호 (역삼동, 역삼동658빌딩)",서울특별시 강남구 역삼동 658번지 역삼동658빌딩,역삼1동,커피숍,23.40,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피


In [ ]:
data_clean2[data_clean2['업소명'].str.contains("주식회사")]

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
268,2019,20190426.0,주식회사 매머드커피랩,"서울특별시 강남구 영동대로 513, 코엑스 A Hall 1층 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,삼성2동,커피숍,18.00,NaN,NaN,NaN,NaN,NaN,지상,비대상,20190520.0,자진폐업,매머드커피
634,2018,20191120.0,주식회사 지함 이디야커피 송파위례점,"서울특별시 송파구 위례광장로 188, 아이온스퀘어 135호, 136호 (장지동)",서울특별시 송파구 장지동 881번지 아이온스퀘어,장지동,기타 휴게음식점,85.37,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
635,2018,20180531.0,주식회사 지함 이디야커피 송파위례점,"서울특별시 송파구 위례광장로 188, 아이온스퀘어 135호, 136호 (장지동)",서울특별시 송파구 장지동 881번지 아이온스퀘어,장지동,기타 휴게음식점,85.37,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1163,2016,20180208.0,투썸플레이스주식회사 투썸플레이스CJ헬로점,"서울특별시 마포구 월드컵북로56길 19, (상암동, 드림타워 6층일부)",서울특별시 마포구 상암동 1595번지 1호 드림타워 6층일부,상암동,커피숍,39.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1168,2016,20170922.0,투썸플레이스주식회사 투썸플레이스CJ헬로점,"서울특별시 마포구 월드컵북로56길 19, (상암동, 드림타워 6층일부)",서울특별시 마포구 상암동 1595번지 1호 드림타워 6층일부,상암동,커피숍,39.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1172,2016,20190701.0,투썸플레이스주식회사 투썸플레이스CJ헬로점,"서울특별시 마포구 월드컵북로56길 19, (상암동, 드림타워 6층일부)",서울특별시 마포구 상암동 1595번지 1호 드림타워 6층일부,상암동,커피숍,39.60,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
1290,2016,20160122.0,메가푸드앤서비스주식회사노량진지점,"서울특별시 동작구 노량진로 140, 지상5층 (노량진동)",서울특별시 동작구 노량진동 57번지 1호,노량진제1동,편의점,12.26,5.0,5.0,NaN,NaN,NaN,지상2층이상,비대상,20170310.0,자진폐업,nan
2551,2012,20121019.0,주식회사 메가젠임플란트 메가젠타워지점,"서울특별시 강남구 선릉로 607, 지상1층 (논현동, 메가젠타워)",서울특별시 강남구 논현동 268번지 15호 메가젠타위 1층,논현2동,커피숍,218.80,NaN,NaN,NaN,NaN,NaN,지상,비대상,20130614.0,자진폐업,nan
3020,2009,20180208.0,투썸플레이스 주식회사 투썸플레이스 상암E&M점,"서울특별시 마포구 상암산로 66, (상암동, CJ E&M센터 1층)",서울특별시 마포구 상암동 1606번지,상암동,커피숍,117.09,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan
3023,2009,20170922.0,투썸플레이스 주식회사 투썸플레이스 상암E&M점,"서울특별시 마포구 상암산로 66, (상암동, CJ E&M센터 1층)",서울특별시 마포구 상암동 1606번지,상암동,커피숍,117.09,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,nan


## 6. 고가 프랜차이즈
- 스타벅스
- 투썸플
- 할리스
- 파스쿠
- 폴바셋 ???


### (1) 스타벅스





In [ ]:
data_clean2[data_clean2['업소명'].str.contains('스타벅스')]
#.shape #56

NameError: name 'data_clean2' is not defined

## 7. 중가 프랜차이즈

## 8.데이터셋 분리
-  저가/중가/고가-프랜차이즈
- 그외 카페 (거짓 메가 포함되어있음)

### 저가-프랜차이즈 행만 뽑기
[cafe_low]
- 메가커피
- 컴포즈커피
- 매머드커피
- 더벤티
- 빽다방

In [ ]:
cafe_low = data_clean2[data_clean2['프랜차이즈명'].isin(['메가커피', '컴포즈커피', '매머드커피', '더벤티', '빽다방'])]

In [ ]:
cafe_low.shape
# 서울 저가-프랜차이즈카페 (5종) : 1898개

(1898, 18)

In [ ]:
cafe_low.isna().sum()

,0
년도,0
영업자시작일,0
업소명,0
소재지도로명,0
소재지지번,2
행정동명,2
업태명,20
총면적,0
지상_부터,823
지상_까지,847


### 고가-프랜차이즈 행만 뽑기
[cafe_high]

### 중가-프랜차이즈 행만 뽑기
[cafe_middle]

### 그 외 카페 데이터셋 만들기
- 예를 들어, 코엑스에는 카페 정말 많음 > 유동인구가 많지만 다른 카페들이 많기 때문에 저가-프랜차이즈 구매율에 영향 줄 확률 높음. 고려해야됨

## 6. 업소명 정리

**- 도로명 채우는 코드 추가 후 > 다시 코드 돌려야됨**
- '도로명세부' 변수 생성
- 같은 프랜차이즈 중에서 겹치는 거 있는지 확인

In [ ]:
cafe_low.head()

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명
0,2019,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,컴포즈커피
16,2019,20190410.0,매머드 익스프레스 청계다동점,"서울특별시 중구 다동길 43, 한외빌딩 1층 118,119,120호 (다동)",서울특별시 중구 다동 70번지 한외빌딩,명동,커피숍,33.33,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피
27,2019,20190531.0,메가엠지씨커피 동대문점,"서울특별시 중구 장충단로13길 13, 1층 (을지로6가)",서울특별시 중구 을지로6가 18번지 97호,광희동,커피숍,15.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피
30,2019,20190401.0,매머드 익스프레스 동대문점,"서울특별시 중구 장충단로13길 43, 1층 24-1호 (을지로6가)",서울특별시 중구 을지로6가 18번지 187호,광희동,커피숍,6.45,1.0,1.0,NaN,NaN,1.0,지상,비대상,NaN,NaN,매머드커피
35,2019,20191112.0,웨일즈커피 메가박스 성수점,"서울특별시 성동구 왕십리로 50, 3층 (성수동1가)",서울특별시 성동구 성수동1가 656번지 335호,성수1가제1동,커피숍,157.50,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피


In [ ]:
# '소재지도로명'을 공백으로 나누고, 마지막 문자열을 '도로명세부'라는 새로운 변수에 할당
cafe_low['도로명세부'] = cafe_low['소재지도로명'].apply(lambda x: x.split()[-1])

<ipython-input-68-2c71b8b737b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_low['도로명세부'] = cafe_low['소재지도로명'].apply(lambda x: x.split()[-1])


In [ ]:
cafe_low

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명,도로명세부
0,2019,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,컴포즈커피,(계동)
16,2019,20190410.0,매머드 익스프레스 청계다동점,"서울특별시 중구 다동길 43, 한외빌딩 1층 118,119,120호 (다동)",서울특별시 중구 다동 70번지 한외빌딩,명동,커피숍,33.33,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피,(다동)
27,2019,20190531.0,메가엠지씨커피 동대문점,"서울특별시 중구 장충단로13길 13, 1층 (을지로6가)",서울특별시 중구 을지로6가 18번지 97호,광희동,커피숍,15.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피,(을지로6가)
30,2019,20190401.0,매머드 익스프레스 동대문점,"서울특별시 중구 장충단로13길 43, 1층 24-1호 (을지로6가)",서울특별시 중구 을지로6가 18번지 187호,광희동,커피숍,6.45,1.0,1.0,NaN,NaN,1.0,지상,비대상,NaN,NaN,매머드커피,(을지로6가)
35,2019,20191112.0,웨일즈커피 메가박스 성수점,"서울특별시 성동구 왕십리로 50, 3층 (성수동1가)",서울특별시 성동구 성수동1가 656번지 335호,성수1가제1동,커피숍,157.50,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피,(성수동1가)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,2024,20240104.0,메가엠지씨커피 청량리롯데캐슬점,"서울특별시 동대문구 답십리로 27, 1층 109호 (전농동, 청량리역 롯데캐슬 SK...",서울특별시 동대문구 전농동 620번지 47호 청량리역 롯데캐슬 SKY-L65,전농제1동,커피숍,29.70,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,메가커피,SKY-L65)
5536,2024,20240104.0,컴포즈커피 양재이마트점,"서울특별시 서초구 매헌로 16, 하이브랜드 지하1층 리빙관 지102호 (양재동)",서울특별시 서초구 양재동 215번지 하이브랜드 지하1층 리빙관 지102호,양재제2동,기타 휴게음식점,27.81,NaN,NaN,1.0,1.0,NaN,지하,비대상,NaN,NaN,컴포즈커피,(양재동)
5537,2024,20240104.0,메가엠지씨커피 신림신원점,"서울특별시 관악구 신원로 13-1, 1층 (신림동)",서울특별시 관악구 신림동 1604번지 41호,신원동,커피숍,36.58,1.0,1.0,NaN,NaN,5.0,NaN,비대상,NaN,NaN,메가커피,(신림동)
5539,2024,20240103.0,컴포즈커피 우장산숲아이파크점,"서울특별시 강서구 강서로 200, 섬유유통노련회관 1층 좌측호 (화곡동)",서울특별시 강서구 화곡동 1031번지 1호 섬유유통노련회관,우장산동,커피숍,127.05,1.0,1.0,NaN,NaN,6.0,지상,비대상,NaN,NaN,컴포즈커피,(화곡동)


In [ ]:
cafe_low[cafe_low['소재지도로명']==' ']

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명,도로명세부


In [ ]:
# '프랜차이즈명'을 기준으로 '행정동명'의 개수를 계산
cafe_low.groupby('프랜차이즈명')['행정동명'].nunique()

,행정동명
프랜차이즈명,
더벤티,112
매머드커피,165
메가커피,308
빽다방,202
컴포즈커피,230


In [ ]:
cafe_low.groupby('프랜차이즈명')['행정동명'].value_counts() # 중복 있으니까, 업소명 세부에 행정동명으로 하면 안됨

프랜차이즈명  행정동명 
더벤티     등촌제3동    10
        창신제1동     5
        구로제3동     5
        목5동       3
        중앙동       3
                 ..
컴포즈커피   수유2동      1
        수유3동      1
        수유제1동     1
        수유제2동     1
        흑석동       1
Name: count, Length: 1017, dtype: int64

In [ ]:
# '프랜차이즈명'을 기준으로 '행정동명'의 개수를 계산
cafe_low.groupby('프랜차이즈명')['도로명세부'].nunique()

,도로명세부
프랜차이즈명,
더벤티,104
매머드커피,202
메가커피,301
빽다방,187
컴포즈커피,208


In [ ]:
cafe_low.groupby('프랜차이즈명')['도로명세부'].value_counts() # 얘도 겨치는게 많네..

프랜차이즈명  도로명세부      
더벤티     엔씨백화점)         8
        (신림동)          6
        (창신동)          5
        (구로동)          5
        (목동)           4
                      ..
컴포즈커피   (염창동)          1
        (온수동)          1
        (용두동)          1
        (용문동)          1
        홍제역해링턴플레이스)    1
Name: count, Length: 1002, dtype: int64

굳이 업소명 자세하게 변경하지 말까?

## 7, '3년 이내 폐업여부' 변수 생성

In [ ]:
cafe_low

In [ ]:
file_path = '/content/drive/MyDrive/cafe_low.csv'
cafe_low.to_csv(file_path, index=False)

In [ ]:
cafe_low

,년도,영업자시작일,업소명,소재지도로명,소재지지번,행정동명,업태명,총면적,지상_부터,지상_까지,지하_부터,지하_까지,총층수,업소위치,모범음식점여부,폐업일자,폐업구분,프랜차이즈명,도로명세부
0,2019,20191118.0,컴포즈 커피,"서울특별시 종로구 율곡로 75, 현대빌딩 지하1층 (계동)",서울특별시 종로구 계동 140번지 2호 현대빌딩,가회동,커피숍,16.44,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,컴포즈커피,(계동)
16,2019,20190410.0,매머드 익스프레스 청계다동점,"서울특별시 중구 다동길 43, 한외빌딩 1층 118,119,120호 (다동)",서울특별시 중구 다동 70번지 한외빌딩,명동,커피숍,33.33,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,매머드커피,(다동)
27,2019,20190531.0,메가엠지씨커피 동대문점,"서울특별시 중구 장충단로13길 13, 1층 (을지로6가)",서울특별시 중구 을지로6가 18번지 97호,광희동,커피숍,15.00,1.0,1.0,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피,(을지로6가)
30,2019,20190401.0,매머드 익스프레스 동대문점,"서울특별시 중구 장충단로13길 43, 1층 24-1호 (을지로6가)",서울특별시 중구 을지로6가 18번지 187호,광희동,커피숍,6.45,1.0,1.0,NaN,NaN,1.0,지상,비대상,NaN,NaN,매머드커피,(을지로6가)
35,2019,20191112.0,웨일즈커피 메가박스 성수점,"서울특별시 성동구 왕십리로 50, 3층 (성수동1가)",서울특별시 성동구 성수동1가 656번지 335호,성수1가제1동,커피숍,157.50,NaN,NaN,NaN,NaN,NaN,지상,비대상,NaN,NaN,메가커피,(성수동1가)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,2024,20240104.0,메가엠지씨커피 청량리롯데캐슬점,"서울특별시 동대문구 답십리로 27, 1층 109호 (전농동, 청량리역 롯데캐슬 SK...",서울특별시 동대문구 전농동 620번지 47호 청량리역 롯데캐슬 SKY-L65,전농제1동,커피숍,29.70,NaN,NaN,NaN,NaN,NaN,NaN,비대상,NaN,NaN,메가커피,SKY-L65)
5536,2024,20240104.0,컴포즈커피 양재이마트점,"서울특별시 서초구 매헌로 16, 하이브랜드 지하1층 리빙관 지102호 (양재동)",서울특별시 서초구 양재동 215번지 하이브랜드 지하1층 리빙관 지102호,양재제2동,기타 휴게음식점,27.81,NaN,NaN,1.0,1.0,NaN,지하,비대상,NaN,NaN,컴포즈커피,(양재동)
5537,2024,20240104.0,메가엠지씨커피 신림신원점,"서울특별시 관악구 신원로 13-1, 1층 (신림동)",서울특별시 관악구 신림동 1604번지 41호,신원동,커피숍,36.58,1.0,1.0,NaN,NaN,5.0,NaN,비대상,NaN,NaN,메가커피,(신림동)
5539,2024,20240103.0,컴포즈커피 우장산숲아이파크점,"서울특별시 강서구 강서로 200, 섬유유통노련회관 1층 좌측호 (화곡동)",서울특별시 강서구 화곡동 1031번지 1호 섬유유통노련회관,우장산동,커피숍,127.05,1.0,1.0,NaN,NaN,6.0,지상,비대상,NaN,NaN,컴포즈커피,(화곡동)


[해야할 것]
- ㅍ